In [ ]:
import os
import random
from scipy import ndarray

# image processing library
import skimage as sk
from skimage import transform
from skimage import util
from skimage import io

def random_rotation(image_array: ndarray):
    # pick a random degree of rotation between 25% on the left and 25% on the right
    random_degree = random.uniform(-25, 25)
    return sk.transform.rotate(image_array, random_degree)

def random_noise(image_array: ndarray):
    # add random noise to the image
    return sk.util.random_noise(image_array)

def horizontal_flip(image_array: ndarray):
    # horizontal flip doesn't need skimage, it's easy as flipping the image array of pixels !
    return image_array[:, ::-1]

# dictionary of the transformations we defined earlier
available_transformations = {
    'rotate': random_rotation,
    'noise': random_noise,
    'horizontal_flip': horizontal_flip
}

folder_path = '/Users/rohitgulve/Documents/Applied_Machine_Learning/data3a/training/02-moderate'
num_files_desired = 2000

# find all files paths from the folder
images = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

num_generated_files = 0
while num_generated_files <= num_files_desired:
    # random image from the folder
    image_path = random.choice(images)
    # read image as an two dimensional array of pixels
    image_to_transform = sk.io.imread(image_path)
    # random num of transformation to apply
    num_transformations_to_apply = random.randint(1, len(available_transformations))

    num_transformations = 0
    transformed_image = None
    while num_transformations <= num_transformations_to_apply:
        # random transformation to apply for a single image
        key = random.choice(list(available_transformations))
        transformed_image = available_transformations[key](image_to_transform)
        num_transformations += 1

new_file_path = '%s/augmented_image_%s.jpg' % (folder_path, num_generated_files)

# write image to the disk
io.imsave(new_file_path, transformed_image)
num_generated_files += 1

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

# Create an instance of the ImageDataGenerator class
datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

# Use the flow_from_directory method to read the images from the "images" folder
# and apply the data augmentation techniques
generator = datagen.flow_from_directory(
        '/Users/rohitgulve/Documents/Applied_Machine_Learning/data3a/training/02-moderate',
        target_size=(150, 150),
        batch_size=32,
        class_mode='binary')

In [ ]:
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

# Define data augmentation parameters
datagen = ImageDataGenerator(
        rotation_range=20, # randomly rotate images within 20 degrees
        width_shift_range=0.2, # randomly shift images horizontally by 20% of the width
        height_shift_range=0.2, # randomly shift images vertically by 20% of the height
        shear_range=0.2, # randomly apply shearing transformations
        zoom_range=0.2, # randomly zoom images
        horizontal_flip=True, # randomly flip images horizontally
        fill_mode='nearest') # fill in any empty pixels with the nearest value

# Specify the directory containing the images to be augmented
input_dir = '/Users/rohitgulve/Documents/Applied_Machine_Learning/data3a/training/02-moderate'

# Loop through each image in the directory and apply data augmentation
for filename in os.listdir(input_dir):
    img = load_img(os.path.join(input_dir, filename))  # Load the image
    x = img_to_array(img)  # Convert the image to a numpy array
    x = x.reshape((1,) + x.shape)  # Reshape the array to (1, height, width, channels)

    # Generate batches of augmented images and save them to the output directory
    i = 0
    for batch in datagen.flow(x, batch_size=1, save_to_dir=input_dir, save_prefix='aug_', save_format='jpeg'):
        i += 1
        if i >= 5:  # Generate 5 augmented images for each input image
            break
